In [1]:
#Imorting the Libraries
import cv2 as cv
import time
import numpy as np
import mediapipe as mp
from playsound import playsound

In [ ]:
#Getting the required functions form mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [ ]:
image1 = cv.imread('GreenLight.png')  ##Reading the Green light Image
image2 = cv.imread('RedLight.png')   ##Reading the Red light Image

currImage = image1

In [ ]:
#Calculating the area of the all landmarks 
def calc_sum(landmarkList):
    
    cur_sum = 0
    for i in range(0,33):
        cur_sum+=(landmarkList[i].x * 480)
    
    return cur_sum

In [ ]:
#Caculating the distance between the legs to progress the movement
def calc_dist(landmarkList):
    return (landmarkList[28].y*640 - landmarkList[24].y*640)

In [ ]:
#Checking the visibility of person
def isVisible(landmarkList):
    if (landmarkList[28].visibility > 0.6) and (landmarkList[24].visibility > 0.6):
        return 1
    return 0

In [ ]:
cPos = 0 #Position of the person in the game
startTime = 0  #Starting time after doll rotated
endTime = 0 #End time before doll rotate himsel
userSum = 0 #Storing all landmarks area
dur = 0 #Duration of total game
isAlive = 1 #Checking if alive or not in the game
isInit = False
cStart, cEnd = 0,0
isCinit = False
tempSum = 0 #Maintaining the tempsum of all landmarks
winner = 0 #Maintaining the Winner status
inFrame = 0 #Checking in frame or not
inFramecheck = False
thresh = 180 #Maintaining the threshold of movement

capture = cv.VideoCapture(0)
while True:
    
    ret,frame = capture.read()
    RGBframe = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    results = mp_pose.Pose().process(RGBframe)
    frame = cv.blur(frame,(5,5))
    mp_drawing.draw_landmarks(frame,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             )
    
    if not(inFramecheck):
        try:
            if isVisible(results.pose_landmarks.landmark):
                inFrame=1
                inFramecheck=True
            else:
                inFrame = 0
        except:
            print("You are not visible at all")
            
    if inFrame==1:
        if not(isInit):
            playsound('DollMusic.mp3')
            currImage = image1
            startTime = time.time()
            endTime = startTime
            isInit = True
            
        if (endTime - startTime) <=5:
            try:
                m = calc_dist(results.pose_landmarks.landmark)
                if m < thresh:
                    cPos +=1
                
                print("Current Progress is:", cPos)
            except:
                print("Not Visible")
            
            endTime = time.time()
        else:
            if cPos>= 100:
                print("Winner")
                winner=1
            else:
                if not(isCinit):
                    isCinit = True
                    cStart = time.time()
                    cEnd = cStart
                    currImage = image2
                    playsound('DollScan.mp3')
                    userSum = calc_sum(results.pose_landmarks.landmark)
                
                if (cEnd - cStart) <=5:
                    tempSum = calc_sum(results.pose_landmarks.landmark)
                    cEnd = time.time()
                    if abs(tempSum - userSum) > 150:
                        print("Dead", abs (tempSum - userSum))
                        isAlive = 0
                else:
                    isInit = False
                    isCinit = False
        
        cv.circle(currImage, ((55+6*cPos),280),15,(0,0,255),-1)
        
        mainWin = np.concatenate((cv.resize(frame,(800,400)),currImage),axis=0)
        cv.imshow("Squid Game",mainWin)
    else:
        cv.putText(frame,"Please Stay in a position where your body fully visible",(20,200), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 4)
        cv.imshow("Frame",frame)
    
    if (cv.waitKey(1) & 0xFF == ord('d')) or isAlive == 0 or winner == 1:
        break
capture.release()
cv.destroyAllWindows()

if isAlive == 0:
    cv.putText(frame, "You are Dead", (50,200), cv.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 4)
    cv.imshow("Squid Game",frame)
if winner == 1:
    cv.putText(frm, "You are Winner", (50,200), cv.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 4)
    cv.imshow("Squid Game",frame)

cv.waitKey(0)

In [ ]:
capture.release()


